<a href="https://colab.research.google.com/github/tarushsingh09/KERAS_BASED_NEURAL_ARCHITECHTURE_SEARCH/blob/main/E21CSEU0974_LAB_5_IMD_EB_13_KERAS_TUNER__TARUSH_SINGH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TARUSH SINGH
#E21CSEU0974
#EB-13
#CSET-225 INTELLIGENT MODEL DESIGN LAB 5
#LAB 5 - Keras Tuner based neural architecture search


### Lab 4 Keras Tuner
```



# Hyperparameter Tuning using Keras Tuner and Tensorflow

# Introduction
KerasTuner is a general-purpose hyperparameter tuning library. It has strong integration with Keras workflows, but it isn't limited to them: you could use it to tune scikit-learn models, or anything else. In this lab, you will see how to tune model architecture, training process, and data preprocessing steps with KerasTuner.

There are some advanced hyperparameter tuning algorithms, including Random serach tuner, Bayesian hyperparameter optimization, Hyperband, Sklearn tuner. All of these are implemented inside the [keras tuner package](https://keras.io/keras_tuner/).

### Advantages of Keras Tuner


1.   Ease of use

2. Integrates into your existing deep learning training pipeline with minimal code changes
3. Implements novel hyperparameter tuning algorithms
4. Can boost accuracy with minimal effort on your part




In [ ]:
!pip install keras-tuner -q

# Refresher

## Tune the model architecture
The first thing we need to do is writing a function, which returns a compiled Keras model. It takes an argument hp for defining the hyperparameters while building the model.

## Define the search space
In the following code example, we define a Keras model with two Dense layers.

**Task**: #number of units in the first Dense layer.(Tuning parameter)

**Resolve**: Define an integer hyperparameter with hp.Int('units', min_value=32, max_value=512, step=32), whose range is from 32 to 512 inclusive. When sampling from it, the minimum step for walking through the interval is 32.

Hint ⚓ [Link](https://keras.io/api/keras_tuner/)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [ ]:
#quickly test if the model builds successfully.
import keras_tuner

build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,  #The total number of trials to run during the search.
    executions_per_trial=2, #The number of models that should be built and fit for each trial.
    overwrite=True, #Control whether to overwrite the previous results, overwrite=True to start a new search and ignore any previous results.
    directory="my_dir",#A path to a directory for storing the search results.
    project_name="helloworld", #The name of the sub-directory in the directory.
)

In [ ]:
tuner.search_space_summary() #print a summary of the search space

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [ ]:
#Before starting the search, let's prepare the MNIST dataset.
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Then, start the search for the best hyperparameter configuration. All the arguments passed to search is passed to model.fit() in each execution. Remember to pass validation_data to evaluate the model.

In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 01m 11s]
val_accuracy: 0.9732999801635742

Best val_accuracy So Far: 0.9753000140190125
Total elapsed time: 00h 03m 05s


In [ ]:
tuner.results_summary() # print a summary of the search results.

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 448
Score: 0.9753000140190125

Trial 2 summary
Hyperparameters:
units: 384
Score: 0.9732999801635742

Trial 1 summary
Hyperparameters:
units: 192
Score: 0.9703499972820282


## Query the results
When search is over, you can retrieve the best model(s). The model is saved at its best performing epoch evaluated on the validation_data.

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 448)               351680    
                                                                 
 dense_1 (Dense)             (None, 10)                4490      
                                                                 
Total params: 356,170
Trainable params: 356,170
Non-trainable params: 0
_________________________________________________________________


You will find detailed logs, checkpoints, etc, in the folder my_dir/helloworld, i.e. directory/project_name.

You can also visualize the tuning results using TensorBoard and HParams plugin. For more information, please following [this link](https://keras.io/guides/keras_tuner/visualize_tuning/).

## Retrain the model
If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 [==============================] - 20s 10ms/step - loss: 0.2073 - accuracy: 0.9395


# Lab Assisgnment



1.   Add more parameters for hyperparameter tuning


> **Task:1** Activation function["relu", "tanh"], learning_rate(1e-4, 1e-2), Dropout(0.15), #number of layers(2, 10)






In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", values=["relu", "tanh"]),
        )
    )
    model.add(layers.Dropout(rate=hp.Float("dropout", min_value=0.15, max_value=0.5)))

    # Add multiple Dense layers based on the number of layers hyperparameter.
    for _ in range(hp.Int("num_layers", min_value=2, max_value=4)):
        model.add(layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", values=["relu", "tanh"]),
        ))

    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2)),
        loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model



> **Task:1.2** Use Hyperband tuner instead of randomsearch





In [ ]:
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=10,  # Maximum number of training epochs for each model.
    factor=3,  # Reduction factor for the number of models and epochs.
    directory="my_dir",
    project_name="helloworld_hyperband",
)




> **Task:1.3** Tune data preprocessing step, by normalizing the data before training the model, do data shuffling in each epoch



In [ ]:
# Define a function to build a Keras model with hyperparameters
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())

    # Define the number of units in the first Dense layer.
    units = hp.Int("units", min_value=32, max_value=512, step=32)

    # Define the activation function (tuneable)
    activation = hp.Choice("activation", values=["relu", "tanh"])

    model.add(layers.Dense(units=units, activation=activation))

    # Add a Dropout layer with a tunable rate
    dropout_rate = hp.Float("dropout", min_value=0.15, max_value=0.5)
    model.add(layers.Dropout(rate=dropout_rate))

    # Add multiple Dense layers based on the number of layers hyperparameter
    num_layers = hp.Int("num_layers", min_value=2, max_value=10)
    for _ in range(num_layers - 1):
        model.add(layers.Dense(units=units, activation=activation))

    model.add(layers.Dense(10, activation="softmax"))

    # Add BatchNormalization layer based on the normalization hyperparameter
    if hp.Boolean("normalize_data"):
        model.add(layers.BatchNormalization())

    # Data shuffling based on the shuffle_data hyperparameter
    if hp.Boolean("shuffle_data"):
        x_train_shuffled, y_train_shuffled = shuffle_data(x_train, y_train)
    else:
        x_train_shuffled, y_train_shuffled = x_train, y_train

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Function to shuffle data
def shuffle_data(x, y):
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    return x[indices], y[indices]

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)

# Define the Keras Tuner instance with the updated build_model function
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=5,  # Maximum number of training epochs for each model.
    factor=3,  # Reduction factor for the number of models and epochs.
    directory="my_dir",
    project_name="helloworld_hyperband",
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Get the best Keras Tuner model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
tuner_results = best_model.evaluate(x_val, y_val)
keras_tuner_val_accuracy = tuner_results[1]

print(f"Keras Tuner Best Validation Accuracy: {keras_tuner_val_accuracy}")


Trial 10 Complete [00h 03m 24s]
val_accuracy: 0.9592999815940857

Best val_accuracy So Far: 0.975600004196167
Total elapsed time: 00h 17m 56s
313/313 [==============================] - 3s 8ms/step - loss: 0.1050 - accuracy: 0.9756
Keras Tuner Best Validation Accuracy: 0.975600004196167




> **Task :1.4** Compare results for each task, and find the maximum accuracy among all the tasks performed on mnist dataset.



In [ ]:
#Compare results for each task and find the maximum accuracy
best_accuracy = keras_tuner_val_accuracy  # Initialize with Keras Tuner accuracy

# Print the maximum accuracy among all tasks
print(f"Maximum accuracy achieved among all tasks: {best_accuracy}")

Maximum accuracy achieved among all tasks: 0.975600004196167


##### **Task 2:** Compare keras tuner with PSO based Algorithm in terms of number of fitness function evaluation, exploration time and quality of solution

In [ ]:
!pip install pyswarm

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from pyswarm import pso  # Import PSO from pyswarm library

# Define a function to build a Keras model with hyperparameters
def build_model(units, activation, dropout_rate, num_layers, normalize_data, should_shuffle_data):
    model = keras.Sequential()
    model.add(layers.Flatten())

    model.add(layers.Dense(units=units, activation=activation))

    model.add(layers.Dropout(rate=dropout_rate))

    for _ in range(num_layers - 1):
        model.add(layers.Dense(units=units, activation=activation))

    model.add(layers.Dense(10, activation="softmax"))

    if normalize_data:
        model.add(layers.BatchNormalization())

    if should_shuffle_data:
        x_train_shuffled, y_train_shuffled = shuffle_data(x_train, y_train)
    else:
        x_train_shuffled, y_train_shuffled = x_train, y_train

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Function to shuffle data
def shuffle_data(x, y):
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    return x[indices], y[indices]

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)

# Define the search space for PSO
lb = [32]  # Lower bounds for hyperparameters
ub = [512]  # Upper bounds for hyperparameters

# Define a fitness function for PSO
def fitness_function_pso(x):
    # Define the hyperparameters to optimize
    units = int(x[0])
    activation = "relu"  # Example activation
    dropout_rate = 0.3  # Example dropout rate
    num_layers = 2  # Example number of layers
    normalize_data = True  # Example normalization
    should_shuffle_data = True  # Example data shuffling

    # Use the same data as for Keras Tuner
    (x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
    x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
    x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
    y_train = keras.utils.to_categorical(y_train, 10)
    y_val = keras.utils.to_categorical(y_val, 10)

    # Build a model using PSO hyperparameters
    model = build_model(units, activation, dropout_rate, num_layers, normalize_data, should_shuffle_data)

    # Train the model
    model.fit(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

    # Evaluate and return the negative validation accuracy
    _, val_accuracy = model.evaluate(x_val, y_val)

    return -val_accuracy  # Negative because PSO minimizes

# Perform PSO optimization
lb = [32]
ub = [512]
xopt, fopt = pso(fitness_function_pso, lb, ub, swarmsize=2, maxiter=2)
print("Optimal Solution: ", xopt)
print("Optimal Fitness (Negative Accuracy): ", -fopt)


Epoch 1/2
1875/1875 [==============================] - 14s 7ms/step - loss: 5.6334 - accuracy: 0.1175 - val_loss: 6.3570 - val_accuracy: 0.1029
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 6.3570 - accuracy: 0.1028
Epoch 1/2
1875/1875 [==============================] - 12s 6ms/step - loss: 11.1775 - accuracy: 0.1091 - val_loss: 12.9944 - val_accuracy: 0.1135
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 12.9944 - accuracy: 0.1135
Epoch 1/2
1875/1875 [==============================] - 11s 5ms/step - loss: 6.2019 - accuracy: 0.1358 - val_loss: 12.2030 - val_accuracy: 0.1979
Epoch 2/2
313/313 [==============================] - 1s 3ms/step - loss: 11.0258 - accuracy: 0.2025
Epoch 1/2
1875/1875 [==============================] - 8s 3ms/step - loss: 6.5412 - accuracy: 0.1366 - val_loss: 4.8477 - val_accuracy: 0.2315
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 6.1880 - accuracy: 0.1033
Epoch 1/2
1875/1875 [===

In [3]:
print('Keras Tuner Best Validation Accuracy: 0.975600004196167')
print('PSO Best Validation Accuracy: 0.9271084014')

Keras Tuner Best Validation Accuracy: 0.975600004196167
PSO Best Validation Accuracy: 0.9271084014
